In [1]:
import os
%pwd

'c:\\Programs\\xray\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Programs\\xray'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: int
    params_label: list

In [6]:
from xray.constants import *
from xray.utils.common import read_yaml, create_directories
from xray.entity.config_entity import (PrepareCallbackConfig
                                       )

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config
        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir, "chest_xray\chest_xray\\test")
        create_directories([
            Path(training.root_dir)
        ])
        
        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_label=params.LABELS
        )
        
        return training_config
    

In [8]:
import os
import time
from urllib import request
from zipfile import ZipFile
from xray import logger
from xray.utils.common import get_size
import tensorflow as tf

In [21]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
        
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
#training imports
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
'''class Training:
    def __init__(self, config: TrainingConfig) -> None:
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )'''

'class Training:\n    def __init__(self, config: TrainingConfig) -> None:\n        self.config = config\n        \n    def get_base_model(self):\n        self.model = tf.keras.models.load_model(\n            self.config.updated_base_model_path\n        )\n    \n    def train_valid_generator(self):\n\n        datagenerator_kwargs = dict(\n            rescale = 1./255,\n            validation_split=0.20\n        )\n\n        dataflow_kwargs = dict(\n            target_size=self.config.params_image_size,\n            batch_size=self.config.params_batch_size,\n            interpolation="bilinear"\n        )\n\n        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(\n            **datagenerator_kwargs\n        )\n\n        self.valid_generator = valid_datagenerator.flow_from_directory(\n            directory=self.config.training_data,\n            subset="validation",\n            shuffle=False,\n            **dataflow_kwargs\n        )\n\n        if self.config.param

In [12]:
import numpy as np
import cv2

In [25]:
class Training:
    def __init__(self, config: TrainingConfig) -> None:
        self.config = config
        
    def get_training_data(self, data_dir):
        image_size = self.config.params_image_size
        labels = self.config.params_label
        data = []
        for label in labels:
            path = os.path.join(data_dir, label)
            class_num = labels.index(label)
            
            for image in os.listdir(path):
                try:
                    img_array = cv2.imread(os.path.join(path,image))
        
                    resized_arr = cv2.resize(img_array, (image_size, image_size))
                    data.append([resized_arr, class_num])
                except Exception as e:
                    print(e)
                    
        return np.array(data)
    
    def process_data(self):
        train_data = self.get_training_data('artifacts/data_ingestion/chest_xray/chest_xray/train')
        test_data = self.get_training_data('artifacts/data_ingestion/chest_xray/chest_xray/test')
        val_data = self.get_training_data('artifacts/data_ingestion/chest_xray/chest_xray/val')

        X_train, y_train = self.process_data_helper(train_data)
        X_test, y_test = self.process_data_helper(test_data)
        X_val, y_val = self.process_data_helper(val_data)

        X_train = X_train / 255
        X_test = X_test / 255
        X_val = X_val / 255

        y_train = y_train.reshape(-1, 1)
        y_test = y_test.reshape(-1, 1)
        y_val = y_val.reshape(-1, 1)

        return X_train, y_train, X_test, y_test, X_val, y_val

    def process_data_helper(self, data):
        X = []
        y = []
        for array, label in data:
            X.append(array)
            y.append(label)
        return np.array(X), np.array(y)
    

In [26]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    # Training
    training_config = config.get_training_config()
    training = Training(config=training_config)
    X_train, y_train, X_test, y_test, X_val, y_val = training.process_data()

    # Print training data
    print("X_train shape:", X_train.shape)
    print("y_train shape:", y_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_test shape:", y_test.shape)
    print("X_val shape:", X_val.shape)
    print("y_val shape:", y_val.shape)
    
except Exception as e:
    raise e

[2023-07-22 12:51:35,174: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-22 12:51:35,183: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-22 12:51:35,184: INFO: common: created directory at: artifacts]
[2023-07-22 12:51:35,190: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-22 12:51:35,192: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-07-22 12:51:35,193: INFO: common: created directory at: artifacts\training]
OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



C:\Users\Sandesh Pargaonkar\AppData\Local\Temp\ipykernel_2536\1478520995.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

X_train shape: (5216, 150, 150, 3)
y_train shape: (5216, 1)
X_test shape: (624, 150, 150, 3)
y_test shape: (624, 1)
X_val shape: (16, 150, 150, 3)
y_val shape: (16, 1)


In [ ]:
try:
    """config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()"""
    #prepare base model
    """config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config= prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()"""
    #training & callback
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e